In [1]:
import numpy as np
import pandas as pd
import json
import pickle
import matplotlib.pyplot as plt
import warnings

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torch.utils.data

import matplotlib.pyplot as plt
import os

In [2]:
'''
Read in saved dataset to use to feed into the model.
Further pre-processing still needs to be done on the state vectors
'''

#data_df = pd.read_pickle('/home/amans/Development/scott/nfl-big-data-bowl-2022/clean_dir/from_csv.pkl')
data_df = pd.read_pickle('/home/amans/Development/scott/nfl-big-data-bowl-2022/clean_dir/teammate_dataset.pkl')


data_df.head()

,season,gameId,playId,playIndex,timeIndex,time,playerId,football_pos,state,next_state,reward,action,next_action
0,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,42515.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[32.17, 38.71, 0.34, 3.44, 3.64, 78.81, 341.2...","[[32.07, 39.05, 0.35, 3.57, 3.48, 76.56, 346.4...",[-0.09],"[[0.1, -0.34, -0.01, -0.13, 0.16, 2.25, -5.16,...","[[-0.21, -0.11, -0.01, -0.16, -0.2, 2.99, -10...."
1,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,43442.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[21.67, 43.81, 0.23, 2.35, 3.25, 145.89, 56.5...","[[21.88, 43.92, 0.24, 2.51, 3.45, 142.9, 67.26...",[-0.09],"[[-0.21, -0.11, -0.01, -0.16, -0.2, 2.99, -10....","[[0.39, -0.34, 0.02, 0.29, -0.37, 0.0, -4.31, ..."
2,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,43497.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[34.23, 31.98, 0.54, 5.36, 3.09, 83.47, 308.5...","[[33.84, 32.32, 0.52, 5.07, 3.46, 83.47, 312.8...",[-0.09],"[[0.39, -0.34, 0.02, 0.29, -0.37, 0.0, -4.31, ...","[[0.21, -0.35, 0.0, -0.07, 0.21, -2.28, -3.53,..."
3,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,46118.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[33.75, 35.52, 0.4, 3.96, 2.34, 52.95, 327.5,...","[[33.54, 35.87, 0.4, 4.03, 2.13, 55.23, 331.03...",[-0.09],"[[0.21, -0.35, 0.0, -0.07, 0.21, -2.28, -3.53,...","[[-0.36, -0.36, 0.0, 0.02, -0.51, -6.83, -8.95..."
4,2018,2018090600,677,0,52,2018-09-07T01:34:34.500,45599.0,"[[11.15, 42.28, 0.17, 2.91, 6.78, 0.0, 0.0, 11...","[[20.92, 38.29, 0.52, 5.16, 5.24, 60.63, 39.12...","[[21.28, 38.65, 0.52, 5.14, 5.75, 67.46, 48.07...",[-0.09],"[[-0.36, -0.36, 0.0, 0.02, -0.51, -6.83, -8.95...","[[0.42, -0.24, 0.01, 0.09, -0.09, -3.73, 2.71,..."


In [3]:
'''
State vector 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''

data_df.loc[0,'state'][0]

array([ 3.21700000e+01,  3.87100000e+01,  3.40000000e-01,  3.44000000e+00,
        3.64000000e+00,  7.88100000e+01,  3.41250000e+02,  3.21700000e+01,
        1.20600000e+01,  3.48810000e+02,  2.51250000e+02, -9.30805168e-02,
       -9.95658585e-01, -7.73349927e-02,  9.97005165e-01,  2.09760000e+01,
        3.56000000e+00,  9.79300000e+00])

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cpu


/home/amans/.local/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [5]:
'''
Create torch dataset
'''

features = torch.zeros(len(data_df)).to(device)
labels = torch.zeros(len(data_df)).to(device)

print("size = ", features.size())

size =  torch.Size([172245])


In [6]:
'''
Function that splits the data into a training, validation, and test set
'''
def split_data(dataset, train_split, seed):
    np.random.seed(seed)
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)

    train_num = int(len(dataset)*train_split)
    val_num = (len(dataset) - int(len(dataset)*train_split))//2

    train_indices = indices[0:train_num]
    val_indices = indices[train_num:train_num+val_num]
    test_indices = indices[train_num+val_num:]

    #check to make sure slices correct
    assert len(dataset) == len(train_indices) + len(val_indices) + len(test_indices)

    #dataset = help.normalize(train_indices, dataset)

    train_data = dataset.iloc[train_indices,:]
    val_data = dataset.iloc[val_indices,:]
    test_data = dataset.iloc[test_indices,:]

    return train_data, val_data, test_data

train_data, val_data, test_data = split_data(data_df, 0.7, 2430)

print(test_data.columns)

print(f"Length of training data: {len(train_data)}")

Index(['season', 'gameId', 'playId', 'playIndex', 'timeIndex', 'time',
       'playerId', 'football_pos', 'state', 'next_state', 'reward', 'action',
       'next_action'],
      dtype='object')
Length of training data: 120571


In [7]:
'''
Predict ['s', 'a', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', 'sin_adj_dir', 'cos_adj_dir']
from 
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''



'''
Define hyperparamters
'''
BATCH_SIZE = 64
state_size = len(data_df.loc[0,'state'][0])
action_size = 8
gamma = 0.99
#max_action = torch.tensor(max_action, dtype=torch.float32).to(device=device)

print(f"State size: {state_size}")
print(f"Action size: {action_size}")
    
'''
First CNN model
cnn_model = nn.Sequential(
    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),
    nn.LeakyReLU(),
    nn.Conv2d(12, 24, (6, 1)),
    nn.LeakyReLU(),
    nn.Conv2d(24, 48, (1,1)),
    nn.Flatten(),
    nn.Linear(48*6*6,action_size)
)

cnn_model.to(device=device)
'''

State size: 18
Action size: 8


'\nFirst CNN model\ncnn_model = nn.Sequential(\n    nn.Conv2d(1, 12, (12,1), stride=1, padding=0),\n    nn.LeakyReLU(),\n    nn.Conv2d(12, 24, (6, 1)),\n    nn.LeakyReLU(),\n    nn.Conv2d(24, 48, (1,1)),\n    nn.Flatten(),\n    nn.Linear(48*6*6,action_size)\n)\n\ncnn_model.to(device=device)\n'

In [ ]:

# lowest val loss for this model was .34. Then starter going up (e.g. ~.88)
cnn_model = nn.Sequential(
    nn.Conv2d(1, 32, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(32, 64, (3,3), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(64,128, (2,2), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Conv2d(128,256, (1,1), stride=1,padding=0),
    nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    #nn.Conv2d(256,512, (3,3), stride=1,padding=0),
    #nn.LeakyReLU(),
    #nn.MaxPool2d((2,2), stride=2),
    nn.Flatten(),
    nn.Linear(13824, 6000),
    nn.LeakyReLU(),
    nn.Linear(6000, 2000),
    nn.LeakyReLU(),
    nn.Linear(2000, 500),
    nn.LeakyReLU(),
    nn.Linear(500, 100),
    nn.LeakyReLU(),
    nn.Linear(100, action_size)
)


# small teammate model ~ 0.3046 val accuracy after 4 epochs, batch=64
# cnn_model = nn.Sequential(
#     nn.Conv2d(1, 6, (2,2), stride=1, padding=0),
#     nn.Flatten(),
#     nn.Linear(924,300), 
#     nn.LeakyReLU(),
#     nn.Linear(300,100), 
#     nn.LeakyReLU(),
#     nn.Linear(100,action_size)
# )


'''
cnn_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(176,100), 
    nn.LeakyReLU(),
    nn.Linear(100,50), 
    nn.LeakyReLU(),
    nn.Linear(50,action_size)
)

cnn_model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(176,60), 
    nn.Dropout(p=0.5),
    nn.LeakyReLU(),
    nn.Linear(60,action_size)
)
'''


cnn_model.to(device=device)

In [ ]:
# define loss function
mse_loss_fn = nn.MSELoss()
# define optimizers
cnn_optimizer = optim.Adam(cnn_model.parameters())

In [ ]:
np.stack(test_data.iloc[0,:]['action'])[0,1:3]

In [ ]:
'''
Given indices, train model on batch
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''
def train_loop(df, indices, model, loss_fn, optimizer, device):
    size = len(indices)
    play = df.iloc[indices, :]
    
    numpy_state = np.stack(play['state'].values)
    batch_size = numpy_state.shape[0]
    numpy_state_input = numpy_state.reshape(batch_size, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
    
    true_state = torch.tensor(numpy_state, dtype=torch.float32).to(device=device)
    true_action = torch.tensor(np.stack(play['action'].values)[:,0,[3,4,7,8,11,12,13,14]], dtype=torch.float32).to(device=device)
    
    # shape true state into matrix to pass into CNN
    #true_state_mod = true_state.reshape(batch_size, -1, len(true_action))
    true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

    model.train()  # put model to training mode

    #compute prediction and loss
    predicted_action = model(true_state_mod)
    
    loss = loss_fn(predicted_action, true_action)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss/size

In [ ]:
def test_loop(test_df, model, loss_fn, device):
    size = len(test_df)
    test_loss = 0
    
    with torch.no_grad():
        for row_index in range(0,len(test_df)):

            play = test_df.iloc[row_index, :]

            true_state = torch.tensor(play['state'], dtype=torch.float32).to(device=device)
            true_action = torch.tensor(play['action'][0,[3,4,7,8,11,12,13,14]], dtype=torch.float32).to(device=device)
            
            # useful for if testing in batches
            #true_state = torch.tensor(np.stack(play['state'].values), dtype=torch.float32).to(device=device)
            #true_action = torch.tensor(np.stack(play['action'].values), dtype=torch.float32).to(device=device)

            numpy_state_input = play['state'].reshape(1, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
                           
            true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

            model.eval()  # put model to eval mode

            #compute prediction and loss
            predicted_action = model(true_state_mod)
  
            true_action = torch.unsqueeze(true_action, 0)

            loss = loss_fn(predicted_action, true_action)
            
            test_loss += loss
        
        test_loss = test_loss/size

    
    return test_loss

In [ ]:
import random

epochs = 4
break_var = False
training_loss_list = []
val_loss_list = []

num_batches = int(len(train_data)/BATCH_SIZE)+1

print(f"Num batches: {num_batches}")

indexes = list(range(0,len(train_data)))

np.random.seed(2430)

for k in range(epochs):
    
    random.shuffle(indexes)
    
    for i in range(num_batches):

        start_index = i*BATCH_SIZE
        end_index = min(len(train_data), (i+1)*BATCH_SIZE)
        
        #indices = list(range(start_index, end_index))
        indices = indexes[start_index:end_index]

        train_loss = train_loop(train_data, indices, cnn_model, mse_loss_fn, cnn_optimizer, device)
        
        training_loss_list.append(train_loss)

        if (i % int(num_batches/2) == 0):# and (i != 0):
            val_loss = test_loop(val_data, cnn_model, F.mse_loss, device)
            val_loss_list.append(val_loss)
            print(f"At epoch {k}, iter {i}: train loss = {train_loss}")
            print(f"At epoch {k}, iter {i}: val loss = {val_loss}")
            
torch.save(cnn_model.state_dict(), 'teammate_model_large.pt')

In [ ]:
'''
Try model predicting entire sequence
'''

In [78]:
'''
same results, not great val loss and then overfit
'''
# cnn_model = nn.Sequential(
#     nn.Conv2d(1, 32, (3,3), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Conv2d(32, 64, (3,3), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Conv2d(64,128, (2,2), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Conv2d(128,256, (1,1), stride=1,padding=0),
#     nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     #nn.Conv2d(256,512, (3,3), stride=1,padding=0),
#     #nn.LeakyReLU(),
#     #nn.MaxPool2d((2,2), stride=2),
#     nn.Flatten(),
#     nn.Linear(13824, 6000),
#     nn.LeakyReLU(),
#     nn.Linear(6000, 2000),
#     nn.LeakyReLU(),
#     nn.Linear(2000, 500),
#     nn.LeakyReLU(),
#     nn.Linear(500, action_size*23)
# )

'''
val loss @ .3 after 4 epochs, .287 after 7 epochs, .286 after 8 epochs
'''
# cnn_model = nn.Sequential(
#     nn.Conv2d(1, 6, (2,2), stride=1, padding=0),
#     nn.Flatten(),
#     nn.Linear(924,400), 
#     nn.LeakyReLU(),
#     nn.Linear(400,action_size*23)
# )

'''
Similar val losses. .359 val loss after 3 epochs, .313 after 4
'''
# cnn_model = nn.Sequential(
#     nn.Conv2d(1, 3, (2,2), stride=1, padding=0),
#     nn.Flatten(),
#     nn.Linear(462,230), 
#     nn.LeakyReLU(),
#     nn.Linear(230,action_size*23)
# )

'''
Similar val losses, .283 val loss after 2 epochs, .281 after 4
'''
# cnn_model = nn.Sequential(
#     nn.BatchNorm2d(1),
#     nn.Conv2d(1, 32, (8,8), stride=1, padding=0),
#     nn.Flatten(),
#     nn.Linear(128*4,230), 
#     nn.LeakyReLU(),
#     nn.Linear(230,action_size*23)
# )

class cnnModel(nn.Module):
    def __init__(self, action_size):
        super(cnnModel, self).__init__()
        
        self.action_size = action_size
        
        self.batch_norm = nn.BatchNorm1d(8)
        self.conv1 = nn.Conv2d(1, 32, (8,8), stride=1, padding=0)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(128*4,230)
        self.leaky = nn.LeakyReLU()
        self.linear2 = nn.Linear(230,action_size*23)
        
        self.drop = nn.Dropout(.5)
        
    # input (batch_size, 1, 23, action_size)
    def forward(self, input_tensor):
        
        #tens_reshape = input_tensor.view(-1, 23, 8)
        initial_shape = input_tensor.shape
        
        batched_input = self.batch_norm(torch.transpose(input_tensor.view(-1, 23, 8), 1,2))
        
        inp = torch.transpose(batched_input, 1, 2).view(initial_shape)
        
        output = self.conv1(inp)
        output = self.flatten(output)
        output = self.drop(output)
        output = self.linear1(output)
        output = self.leaky(output)
        output = self.linear2(output)
        
        return output
cnn_model = cnnModel(action_size)

cnn_model.to(device)

cnnModel(
  (batch_norm): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(1, 32, kernel_size=(8, 8), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=512, out_features=230, bias=True)
  (leaky): LeakyReLU(negative_slope=0.01)
  (linear2): Linear(in_features=230, out_features=184, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
)

In [79]:
from torch.optim.lr_scheduler import LambdaLR
from torch.optim.lr_scheduler import ExponentialLR

# define loss function
mse_loss_fn = nn.MSELoss()
#mse_loss_fn = nn.L1Loss()
# define optimizers
cnn_optimizer = optim.Adam(cnn_model.parameters(), lr=0.005)

#scheduler = ExponentialLR(cnn_optimizer, gamma=0.9)

In [80]:
'''
Given indices, train model on batch
['x','y', 'dis', 's', 'a', 'o', 'dir', 'adj_x', 'adj_y', 'adj_o', 'adj_dir', 'sin_adj_o', 'cos_adj_o', \
'sin_adj_dir', 'cos_adj_dir', 'dist_from_ball_carrier', 'min_teammate_dist', 'min_opponent_dist']
'''
def train_loop_large(df, indices, model, loss_fn, optimizer, device):
    size = len(indices)
    play = df.iloc[indices, :]
    
    numpy_state = np.stack(play['state'].values)
    batch_size = numpy_state.shape[0]
    numpy_state_input = numpy_state.reshape(batch_size, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
    
    true_state = torch.tensor(numpy_state, dtype=torch.float32).to(device=device)
    true_action = torch.tensor(np.stack(play['action'].values)[:,:,[3,4,7,8,11,12,13,14]].reshape(batch_size, -1), dtype=torch.float32).to(device=device)
    
    # shape true state into matrix to pass into CNN
    true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

    model.train()  # put model to training mode

    #compute prediction and loss
    predicted_action = model(true_state_mod)
    
    loss = loss_fn(predicted_action, true_action)

    #Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss/size

In [81]:
def test_loop_large(test_df, model, loss_fn, device):
    size = len(test_df)
    test_loss = 0
    
    with torch.no_grad():
        for row_index in range(0,len(test_df)):

            play = test_df.iloc[row_index, :]

            true_state = torch.tensor(play['state'], dtype=torch.float32).to(device=device)
            true_action = torch.tensor(play['action'][:,[3,4,7,8,11,12,13,14]].reshape(-1), dtype=torch.float32).to(device=device)
            
            # useful for if testing in batches
            #true_state = torch.tensor(np.stack(play['state'].values), dtype=torch.float32).to(device=device)
            #true_action = torch.tensor(np.stack(play['action'].values), dtype=torch.float32).to(device=device)

            numpy_state_input = play['state'].reshape(1, 1, -1, state_size)[:,:,:,[3,4,7,8,11,12,13,14]]
                           
            true_state_mod = torch.tensor(numpy_state_input, dtype=torch.float32).to(device=device)

            model.eval()  # put model to eval mode

            #compute prediction and loss
            predicted_action = model(true_state_mod)
  
            true_action = torch.unsqueeze(true_action, 0)

            loss = loss_fn(predicted_action, true_action)
            
            test_loss += loss
        
        test_loss = test_loss/size

    
    return test_loss

In [82]:
import random

epochs = 8
break_var = False
training_loss_list = []
val_loss_list = []

num_batches = int(len(train_data)/BATCH_SIZE)+1

print(f"Num batches: {num_batches}")

indexes = list(range(0,len(train_data)))

np.random.seed(2430)

for k in range(epochs):
    
    random.shuffle(indexes)
    
    for i in range(num_batches):

        start_index = i*BATCH_SIZE
        end_index = min(len(train_data), (i+1)*BATCH_SIZE)
        
        #indices = list(range(start_index, end_index))
        indices = indexes[start_index:end_index]

        train_loss = train_loop_large(train_data, indices, cnn_model, mse_loss_fn, cnn_optimizer, device)
        
        training_loss_list.append(train_loss)

        if (i % int(num_batches/2) == 0):# and (i != 0):
            #for g in cnn_optimizer.param_groups:
                #g['lr'] = 0.001
                #print(g['lr'])

            val_loss = test_loop_large(val_data, cnn_model, F.mse_loss, device)
            val_loss_list.append(val_loss)
            print(f"At epoch {k}, iter {i}: train loss = {train_loss}")
            print(f"At epoch {k}, iter {i}: val loss = {val_loss}")
            
    for g in cnn_optimizer.param_groups:
        #g['lr'] = 0.001
        g['lr'] /= 10
        print('post scheduler step lr = ', g['lr'])
    
    #scheduler.step()
            
torch.save(cnn_model.state_dict(), 'all_model_large.pt')

Num batches: 1884
At epoch 0, iter 0: train loss = 0.008427238091826439
At epoch 0, iter 0: val loss = 0.6580563187599182
At epoch 0, iter 942: train loss = 0.005429424811154604
At epoch 0, iter 942: val loss = 0.30924633145332336
post scheduler step lr =  0.0005
At epoch 1, iter 0: train loss = 0.005247880704700947
At epoch 1, iter 0: val loss = 0.3010599911212921


KeyboardInterrupt: 